#Conectamos a google drive e importamos librerias

In [ ]:
# Se conecta al Drive:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from numpy.random import seed
seed(1)
import tensorflow
tensorflow.random.set_seed(2)
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import pickle #para guardado de datos

from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import Sequential
from keras.layers.core import Dense, Flatten, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D,Conv2D, AveragePooling2D, BatchNormalization, Dropout, MaxPooling2D,RandomFlip,RandomRotation

import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

# Para importar el Helper
%cd "/content/drive/Shareddrives/Redes neuronales/TP3"
!pip install import-ipynb
import import_ipynb
from Helper import *

/content/drive/Shareddrives/Redes neuronales/TP3
importing Jupyter notebook from Helper.ipynb


# Cargamos los datos

In [ ]:
X_train = np.load("/content/drive/Shareddrives/Redes neuronales/TP3/Dataset original/X_train.npy")
Y_train = np.load("/content/drive/Shareddrives/Redes neuronales/TP3/Dataset original/y_train.npy")
X_test = np.load("/content/drive/Shareddrives/Redes neuronales/TP3/Dataset original/X_test.npy")

# Dividimos entre datasets de entrenamiento y validación

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(X_train, Y_train, test_size = 0.1)

# Paso a float, normalizamos y categorizamos


In [ ]:
x_train = x_train.astype('float32')
x_valid = x_valid.astype('float32')
x_test = X_test.astype('float32')
x_train /= 255
x_valid /= 255
x_test /= 255

In [ ]:
y_train_categorical = to_categorical(y_train,100)
y_val_categorical = to_categorical(y_valid,100)

#Modelo

## Creación de hiperparámetros

In [ ]:
hyperparameters_specs = { 
    'data_augmentation': [True, False],
    'batch_size': [50,100,500,5000], #, 100,500,1000,2000,5000,
    'cant_epochs': [25], #25
}

In [ ]:
hyperparamaters_data_aug = {
    'mode_flip': ['horizontal_and_vertical','horizontal'],
    'factor_rotation': [0.2,0.6],
    'factor_contrast': [0.2,0.6]
}

In [ ]:
# DEFINING HYPERPARAMETERS: hacemos las combinaciones
def get_hyperparameters():
      for batch_size in hyperparameters_specs['batch_size']:
          for cant_epochs in hyperparameters_specs['cant_epochs']:
            for data_aug in hyperparameters_specs['data_augmentation']:
              hyperparameters = {
                  'batch_size':   batch_size,
                  'cant_epochs':  cant_epochs,
                  'data_augmentation': data_aug
              }
            
              yield hyperparameters #gives a generator function, para evitar muchos llamados al código base.

In [ ]:
# DEFINING HYPERPARAMETERS: hacemos las combinaciones
def get_hyperparameters_aug():
      for mode_flip in hyperparamaters_data_aug['mode_flip']:
          for factor_rotation in hyperparamaters_data_aug['factor_rotation']:
            for factor_contrast in hyperparamaters_data_aug['factor_contrast']:
              hyperparameters_aug = {
                  'random_flip':   True,
                  'random_rotation':True,
                  'random_contrast':True,
                  'mode_flip': mode_flip,
                  'factor_rotation': factor_rotation,
                  'factor_contrast': factor_contrast
              }
              
              yield hyperparameters_aug #gives a generator function, para evitar muchos llamados al código base.

## Evaluación de hiperparámetros

In [ ]:
hyperparameters = pd.DataFrame()

for hp in get_hyperparameters():
    df = pd.DataFrame(hp, index=[0])
    hyperparameters = hyperparameters.append(df,ignore_index=True) #ponemos combinaciones

hyperparameters 

,batch_size,cant_epochs,data_augmentation
0,50,25,True
1,50,25,False
2,100,25,True
3,100,25,False
4,500,25,True
5,500,25,False
6,5000,25,True
7,5000,25,False


In [ ]:
for hp in get_hyperparameters_aug():
  for row_index in hyperparameters[hyperparameters['data_augmentation'] == True].index:
    hyperparameters.loc[row_index,'random_flip'] =hp['random_flip']
    hyperparameters.loc[row_index,'random_rotation'] =hp['random_rotation']
    hyperparameters.loc[row_index,'random_contrast'] =hp['random_contrast']
    hyperparameters.loc[row_index,'mode_flip'] =hp['mode_flip']
    hyperparameters.loc[row_index,'factor_rotation'] =hp['factor_rotation']
    hyperparameters.loc[row_index,'factor_contrast'] =hp['factor_contrast']
hyperparameters.fillna('-',inplace=True)

In [ ]:
hyperparameters

,batch_size,cant_epochs,data_augmentation,random_flip,random_rotation,random_contrast,mode_flip,factor_rotation,factor_contrast
0,50,25,True,True,True,True,horizontal,0.6,0.6
1,50,25,False,-,-,-,-,-,-
2,100,25,True,True,True,True,horizontal,0.6,0.6
3,100,25,False,-,-,-,-,-,-
4,500,25,True,True,True,True,horizontal,0.6,0.6
5,500,25,False,-,-,-,-,-,-
6,5000,25,True,True,True,True,horizontal,0.6,0.6
7,5000,25,False,-,-,-,-,-,-


## Data Augmentation, Creación y Entrenamiento del modelo

Solamente se llegaron a probar algunas combinaciones con Alexnet de las que se definieron arriba. Sin embargo, no dieron tan buenos resultados. Vemos que el accuracy del 0,3. Para armar la arquitectura del Alexnet, se armó a mano a través de una funcion que se encuentra en el helper, poniendo cada una de las capas. 

In [ ]:
verbose = 2
monitor='val_acc'
filepath = f'/content/drive/Shareddrives/Redes neuronales/TP3/Modelos entrenados/modelCNN_AlexNet_batch_size_{batch_size}_epocs{epocs}'

In [ ]:
scores = [] #se llegan a correr 4
for index, hp in hyperparameters.iterrows():
  x_train_hp = x_train
  y_train_hp = y_train_categorical
  print(hp.to_dict())
  path_model =f"/content/drive/Shareddrives/Redes neuronales/TP3/Modelos entrenados/emodel_cant_epochs_{hp['cant_epochs']}_batch_size_{hp['batch_size']}_data_augmentation_{hp['data_augmentation']}_random_flip_{hp['random_flip']}_random_rotation_{hp['random_rotation']}_random_contrast_{hp['random_contrast']}_mode_flip_{hp['mode_flip']}_factor_rotation_{hp['factor_rotation']}_factor_contrast_{hp['factor_contrast']}.hdf5"
  
  if hp['data_augmentation']: 
    x_train_hp,y_train_hp = data_augmentation(x_train_hp,y_train_hp,random_flip=hp['random_flip'],mode_flip=hp['mode_flip'],random_rotation=hp['random_rotation'],factor_rotation=hp['factor_rotation'],random_contrast=hp['random_contrast'],factor_contrast=hp['factor_contrast'])

  model = CNN_AlexNet(n_filters=25,loss='categorical_crossentropy',optimizer='Adam', metrics=['accuracy'])
  print(type(hp['cant_epochs']))
  history,model = train(model,monitor,path_model,hp['cant_epochs'],hp['batch_size'],x_train_hp,y_train_hp,x_valid,y_val_categorical,verbose,loss='categorical_crossentropy',optimizer='Adam', metrics=['accuracy'])

  print(max(history.history['val_accuracy']))
  scores.append(max(history.history['val_accuracy'])) 
  print('done.')


hyperparameters['score'] = scores 
hyperparameters.to_csv("/content/drive/Shareddrives/Redes neuronales/TP3/Resultados/Resultados_Alexnet.csv")

{'batch_size': 50, 'cant_epochs': 25, 'data_augmentation': True, 'random_flip': True, 'random_rotation': True, 'random_contrast': True, 'mode_flip': 'horizontal', 'factor_rotation': 0.6, 'factor_contrast': 0.6}
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 32, 32, 25)        9100      
                                                                 
 activation_4 (Activation)   (None, 32, 32, 25)        0         
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 16, 16, 25)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 16, 16, 25)        5650      
                                                                 
 activation_5 (Activation)   (None, 16, 1

In [ ]:
hyperparameters